In [1]:
import pandas as pd
data = pd.read_csv("data_ready_en.csv")
data

,p_recall,delta,ui_language,history_seen,history_correct,session_seen,surface_form,root_form,modifiers_split,history_incorrect,...,pronoun,adjective,adverb,conjunction,determiner,interjection,plural,number,phrase,word_freq
0,1.000000,0.099167,pt,17,17,2,definition,definition,<n><sg>,0,...,0,0,0,0,0,0,0,0,0,0.000038
1,1.000000,0.099167,pt,19,19,3,surface,surface,<n><sg>,0,...,0,0,0,0,0,0,0,0,0,0.000068
2,1.000000,0.099167,pt,21,20,3,scale,scale,<n><sg>,1,...,0,0,0,0,0,0,0,0,0,0.000068
3,0.800000,0.099167,pt,44,36,5,temperature,temperature,<n><sg>,8,...,0,0,0,0,0,0,0,0,0,0.000044
4,1.000000,0.099167,pt,21,20,3,distance,distance,<n><sg>,1,...,0,0,0,0,0,0,0,0,0,0.000071
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4997494,0.800000,0.102222,it,6,4,5,the,the,<det><def><sp>,2,...,0,0,0,0,1,0,0,0,0,0.053700
4997495,0.800000,0.102222,it,4,4,5,eat,eat,<vblex><pres>,0,...,0,0,0,0,0,0,0,0,0,0.000135
4997496,1.000000,0.102222,it,4,4,4,bread,bread,<n><sg>,0,...,0,0,0,0,0,0,0,0,0,0.000032
4997497,0.600000,0.102222,it,3,2,5,drink,drink,<vblex><pres>,1,...,0,0,0,0,0,0,0,0,0,0.000079


In [4]:
print(len(data[data["p_recall"]==1]))

print(len(data[data["p_recall"]<1]))
print(len(data[data["p_recall"]>0.9]))

print((len(data[data["p_recall"]==1]))-(len(data[data["p_recall"]>0.9])))

10768547
2068387
10775485
-6938


In [2]:
data_nonlinguistic = data[["delta","history_seen","history_correct","history_incorrect","passed"]]
data_nonlinguistic

,delta,history_seen,history_correct,history_incorrect,passed
0,0.099167,17,17,0,1
1,0.099167,19,19,0,1
2,0.099167,21,20,1,1
3,0.099167,44,36,8,0
4,0.099167,21,20,1,1
...,...,...,...,...,...
4997494,0.102222,6,4,2,0
4997495,0.102222,4,4,0,0
4997496,0.102222,4,4,0,1
4997497,0.102222,3,2,1,0


In [14]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import SMOTE

In [12]:
X = data_nonlinguistic.drop('passed', axis=1)
y = data_nonlinguistic['passed']
X

,delta,history_seen,history_correct,history_incorrect
0,0.099167,17,17,0
1,0.099167,19,19,0
2,0.099167,21,20,1
3,0.099167,44,36,8
4,0.099167,21,20,1
...,...,...,...,...
4997494,0.102222,6,4,2
4997495,0.102222,4,4,0
4997496,0.102222,4,4,0
4997497,0.102222,3,2,1


In [13]:
print(pd.value_counts(y))

1    4193007
0     804492
Name: passed, dtype: int64


In [15]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_test

,delta,history_seen,history_correct,history_incorrect
2313520,9.200556,16,13,3
3222322,0.076111,138,126,12
1074666,38.472500,2,2,0
4820034,45.882500,21,15,6
4904213,0.051667,5,5,0
...,...,...,...,...
580730,0.066389,6,5,1
704721,0.264722,7,7,0
100968,0.094167,9,7,2
3677723,65.355556,12,11,1


In [17]:

oversampled = SMOTE(random_state=0)
X_train_smote, y_train_smote = oversampled.fit_resample(X_train, y_train)

In [19]:
print(pd.value_counts(y_train))

1    3354072
0     643927
Name: passed, dtype: int64


In [18]:
print(len(X_train_smote))
print(pd.value_counts(y_train_smote))
print(len(y_train_smote))

6708144
1    3354072
0    3354072
Name: passed, dtype: int64
6708144


In [ ]:
clf_default = RandomForestClassifier(n_estimators=64)


In [20]:
from skprog.wrappers import TreesProgressor
from tqdm import tqdm

clf = RandomForestClassifier(n_estimators = 16, max_depth=8, criterion="gini", min_samples_split=64, min_samples_leaf=32, max_features=2, oob_score=True)

In [21]:
clf.fit(X_train_smote, y_train_smote)

c:\users\hresk\pycharmprojects\bakalarka\venv\lib\site-packages\sklearn\ensemble\_forest.py:583: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable OOB estimates.
  warn(


RandomForestClassifier(max_depth=8, max_features=2, min_samples_leaf=32,
                       min_samples_split=64, n_estimators=16, oob_score=True)

In [22]:
y_pred = clf.predict(X_test)

In [23]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.21      0.58      0.31    160565
           1       0.88      0.58      0.70    838935

    accuracy                           0.58    999500
   macro avg       0.54      0.58      0.50    999500
weighted avg       0.77      0.58      0.63    999500



In [24]:
clf.feature_importances_

array([0.24560949, 0.10487378, 0.14587562, 0.50364112])

In [25]:
clf.feature_names_in_

array(['delta', 'history_seen', 'history_correct', 'history_incorrect'],
      dtype=object)

,delta,history_seen,history_correct,history_incorrect
0,7680.454167,6,4,2
1,7680.454167,4,4,0
2,7680.454167,5,4,1
3,7680.454167,6,5,1
4,7680.454167,4,4,0
...,...,...,...,...
12836929,0.102222,6,4,2
12836930,0.102222,4,4,0
12836931,0.102222,4,4,0
12836932,0.102222,3,2,1


In [28]:
from imblearn.under_sampling import RandomUnderSampler
rus = RandomUnderSampler(random_state=42)

X_resampled, y_resampled = rus.fit_resample(X, y)

In [29]:
print(len(X_resampled))
print(pd.value_counts(y_resampled))
print(len(y_resampled))

1608984
0    804492
1    804492
Name: passed, dtype: int64
1608984


In [30]:
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=42)
X_test

,delta,history_seen,history_correct,history_incorrect
1212520,225.018056,3,3,0
368282,32.021111,5,3,2
979148,20.557222,21,17,4
491511,53.195278,2,1,1
1321453,0.075278,34,30,4
...,...,...,...,...
469851,22.215833,1,1,0
919984,0.132500,161,149,12
1178987,94.741944,1,1,0
683817,0.021667,14,10,4


In [37]:
clf = RandomForestClassifier(n_estimators = 16, max_depth=8, criterion="gini", min_samples_split=64, min_samples_leaf=32, max_features=2, oob_score=True)

In [38]:
clf.fit(X_resampled, y_resampled)

c:\users\hresk\pycharmprojects\bakalarka\venv\lib\site-packages\sklearn\ensemble\_forest.py:583: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable OOB estimates.
  warn(


RandomForestClassifier(max_depth=8, max_features=2, min_samples_leaf=32,
                       min_samples_split=64, n_estimators=16, oob_score=True)

In [39]:
y_pred = clf.predict(X_test)

In [43]:
y_pred_prob = clf.predict_proba(X_test)
print(y_pred_prob)

[[0.52208615 0.47791385]
 [0.67153496 0.32846504]
 [0.57388703 0.42611297]
 ...
 [0.46496136 0.53503864]
 [0.66021668 0.33978332]
 [0.58030249 0.41969751]]


In [40]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.58      0.58      0.58    160348
           1       0.58      0.59      0.58    161449

    accuracy                           0.58    321797
   macro avg       0.58      0.58      0.58    321797
weighted avg       0.58      0.58      0.58    321797



In [41]:
clf.feature_importances_

array([0.2222971 , 0.10419768, 0.13394087, 0.53956436])

In [42]:
clf.feature_names_in_

array(['delta', 'history_seen', 'history_correct', 'history_incorrect'],
      dtype=object)